In [1]:
import numpy as np
import sympy as sym
import json
import sympy as sym
import pydae.buildfast as db
from pydae.grid_bpu import bpu

In [2]:
file_path = 'newengland.json'
bpu_obj = bpu(data_input=file_path)

In [3]:
g_list = bpu_obj.dae['g'] 
h_dict = bpu_obj.dae['h_dict']
f_list = bpu_obj.dae['f']
x_list = bpu_obj.dae['x']
params_dict = bpu_obj.dae['params_dict']

sys = {'name':'newengland',
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':bpu_obj.dae['y_ini'],
       'y_run_list':bpu_obj.dae['y_run'],
       'u_run_dict':bpu_obj.dae['u_run_dict'],
       'u_ini_dict':bpu_obj.dae['u_ini_dict'],
       'h_dict':h_dict}

sys = db.system(sys,verbose=True)
db.sys2num(sys,verbose=True)

check_system (time: 0.0
computing jacobians Fx_run,Fy_run  (time: 0.16365528106689453
computing jacobians Gx_run,Gy_run  (time: 6.502557277679443
computing jacobians Fu_run,Gu_run  (time: 34.460084438323975
computing jacobians Fx_ini,Fy_ini  (time: 46.2013144493103
computing jacobians Gx_ini,Gy_ini  (time: 51.14857363700867
computing jacobians Hx_run,Hy_run,Hu_run  (time: 79.05542755126953
end system  (time: 85.01694655418396
computing jac_ini (time: 0.0
computing jac_run (time: 0.1253817081451416
computing jac_trap (time: 0.31370973587036133
generating f_ini_eval string (time: 0.8629999160766602
generating f_run_eval string (time: 20.331060886383057
generating g_ini_eval string (time: 34.94929051399231
generating g_run_eval string (time: 68.25121140480042
generating h_run_eval string (time: 103.80594563484192
generating f_run_gpu string (time: 110.74840903282166
generating h_run_gpu string (time: 127.2387273311615
generating jac_ini_ss_eval string (time: 139.34722256660461
generating 

In [13]:
from sympy import SparseMatrix, Matrix

In [16]:
Gy_ini_sp = SparseMatrix(sys['Gy_ini'])

In [25]:
from sympy.matrices.sparsetools import _doktocsr

Gy_ini_csr = _doktocsr(Gy_ini_sp)

In [33]:
Gy_ini_csr[3]

[170, 170]

In [42]:
def getIndex(s, i): 
  
    # If input is invalid. 
    if s[i] != '(':
        return -1
  
    # Create a deque to use it as a stack. 
    d = deque() 
  
    # Traverse through all elements 
    # starting from i. 
    for k in range(i, len(s)): 
  
        # Pop a starting bracket 
        # for every closing bracket 
        if s[k] == ')': 
            d.popleft() 
  
        # Push all starting brackets 
        elif s[k] == '(': 
            d.append(s[i]) 
  
        # If deque becomes empty 
        if not d: 
            return k 
  
    return -1
  
def arg2np(string,function_name):
    idx_end = 0
    for it in range(3):
        if function_name in string[idx_end:]:
            #print(string[idx_end:])
            idx_ini = string.find(f'{function_name}(',idx_end)+len(f'{function_name}(')
            idx_end = getIndex(string, idx_ini-1)
            string = string.replace(string[idx_ini:idx_end],f'np.array([{string[idx_ini:idx_end]}])')
    else: pass
    return string   

def arg2cp(string,function_name,i_pwise):
    idx_end = 0
    for it in range(3):
        if function_name in string[idx_end:]:
            #print(string[idx_end:])
            idx_ini = string.find(f'{function_name}(',idx_end)+len(f'{function_name}(')
            idx_end = getIndex(string, idx_ini-1)
            string_inline = string.replace(string[idx_ini:idx_end],'pwise[0]').replace(f'{function_name}(','').replace(')','')
            string_pwise = string[idx_ini:idx_end]
    else: pass
    return string_pwise,string_inline 

In [45]:
from collections import deque 

string = str(sys['g'][-3])

ValueError: too many values to unpack (expected 2)

In [47]:
string_pwise,string_inline =  arg2cp(string,'Piecewise',0)
string_pwise_list = string_pwise.split(',')

In [50]:
string_pwise_list

for it in range(int(len(string_pwise_list)/2)):
    string_pwise_list[2*it] = string_pwise_list[2*it][1:] 
    string_pwise_list[2*it+1] = string_pwise_list[2*it+1][:-1] 
    
    
string_pwise_list    

['-V_lim_4',
 ' V_lim_4 < -K_stab_4*(T_1_4*(-x_lead_4 + z_wo_4)/T_2_4 + x_lead_4)',
 '(V_lim_4',
 ' V_lim_4 < K_stab_4*(T_1_4*(-x_lead_4 + z_wo_4)/T_2_4 + x_lead_4)',
 '(K_stab_4*(T_1_4*(-x_lead_4 + z_wo_4)/T_2_4 + x_lead_4)',
 ' True']